In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [172]:
df=pd.read_csv("./Data/incident_event_log.csv")
df.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [173]:
df.shape

(141712, 36)

In [174]:
df.isnull().sum(axis=0) #Calculating NA's in each column

number                     0
incident_state             0
active                     0
reassignment_count         0
reopen_count               0
sys_mod_count              0
made_sla                   0
caller_id                  0
opened_by                  0
opened_at                  0
sys_created_by             0
sys_created_at             0
sys_updated_by             0
sys_updated_at             0
contact_type               0
location                   0
category                   0
subcategory                0
u_symptom                  0
cmdb_ci                    0
impact                     0
urgency                    0
priority                   0
assignment_group           0
assigned_to                0
knowledge                  0
u_priority_confirmation    0
notify                     0
problem_id                 0
rfc                        0
vendor                     0
caused_by                  0
closed_code                0
resolved_by                0
resolved_at   

In [175]:
df.dtypes

number                     object
incident_state             object
active                       bool
reassignment_count          int64
reopen_count                int64
sys_mod_count               int64
made_sla                     bool
caller_id                  object
opened_by                  object
opened_at                  object
sys_created_by             object
sys_created_at             object
sys_updated_by             object
sys_updated_at             object
contact_type               object
location                   object
category                   object
subcategory                object
u_symptom                  object
cmdb_ci                    object
impact                     object
urgency                    object
priority                   object
assignment_group           object
assigned_to                object
knowledge                    bool
u_priority_confirmation      bool
notify                     object
problem_id                 object
rfc           

In [176]:
df.describe()#Estimating the range of values in the dataset

,reassignment_count,reopen_count,sys_mod_count
count,141712.000000,141712.000000,141712.000000
mean,1.104197,0.021918,5.080946
std,1.734673,0.207302,7.680652
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000
50%,1.000000,0.000000,3.000000
75%,1.000000,0.000000,6.000000
max,27.000000,8.000000,129.000000


In [177]:
df.drop(['cmdb_ci', 'problem_id', 'rfc', 'vendor','caused_by','caller_id','opened_by','sys_created_by','sys_updated_by','location','resolved_by','notify','assigned_to','assignment_group'],axis = 1,inplace=True)

In [178]:
df.replace('?', np.NaN, inplace = True)
df.info() #Replacing ? in the dataset ho

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141712 entries, 0 to 141711
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   number                   141712 non-null  object
 1   incident_state           141712 non-null  object
 2   active                   141712 non-null  bool  
 3   reassignment_count       141712 non-null  int64 
 4   reopen_count             141712 non-null  int64 
 5   sys_mod_count            141712 non-null  int64 
 6   made_sla                 141712 non-null  bool  
 7   opened_at                141712 non-null  object
 8   sys_created_at           88636 non-null   object
 9   sys_updated_at           141712 non-null  object
 10  contact_type             141712 non-null  object
 11  category                 141634 non-null  object
 12  subcategory              141601 non-null  object
 13  u_symptom                108748 non-null  object
 14  impact              

In [179]:
#fig, ax = plt.subplots(figsize=(10,10))  
#sns.heatmap(incident.isnull(), cmap='viridis',ax=ax) #Exploring the columns which have most NA values

In [180]:
df.head(5)

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,opened_at,sys_created_at,sys_updated_at,...,subcategory,u_symptom,impact,urgency,priority,knowledge,u_priority_confirmation,closed_code,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,29/2/2016 01:16,29/2/2016 01:23,29/2/2016 01:23,...,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,True,False,code 5,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,29/2/2016 01:16,29/2/2016 01:23,29/2/2016 08:53,...,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,True,False,code 5,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,29/2/2016 01:16,29/2/2016 01:23,29/2/2016 11:29,...,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,True,False,code 5,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,29/2/2016 01:16,29/2/2016 01:23,5/3/2016 12:00,...,Subcategory 170,Symptom 72,2 - Medium,2 - Medium,3 - Moderate,True,False,code 5,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,29/2/2016 04:40,29/2/2016 04:57,29/2/2016 04:57,...,Subcategory 215,Symptom 471,2 - Medium,2 - Medium,3 - Moderate,True,False,code 5,1/3/2016 09:52,6/3/2016 10:00


In [181]:
#remove outliers
df = df[df['incident_state'] != '-100' ]

In [182]:
# replace subtypes of Awaiting incident state into main type of Awaiting
df['incident_state'].replace(['Awaiting User Info','Awaiting Problem','Awaiting Vendor','Awaiting Evidence'],'Awaiting',inplace = True)

In [183]:
df[['priority','incident_state']].groupby('incident_state').agg(['count'])

,priority
,count
incident_state,
Active,38716
Awaiting,15848
Closed,24985
New,36407
Resolved,25751


In [184]:
df['active'] = df['active'].apply(lambda x : 1 if x == 'True' else 0)
df['made_sla'] = df['made_sla'].apply(lambda x : 1 if x == 'True' else 0)
df['knowledge'] = df['knowledge'].apply(lambda x : 1 if x == 'True' else 0)
df['u_priority_confirmation'] = df['u_priority_confirmation'].apply(lambda x : 1 if x == 'True' else 0)
df['priority'] = df['priority'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
df['impact'] = df['impact'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
df['urgency'] = df['urgency'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
reass_count_labels = [1,2,3,4,5]
df['reassignment_count'] = pd.cut(df['reassignment_count'],bins = 5, labels = reass_count_labels)
df = pd.get_dummies(df,columns = ['incident_state'])

In [185]:
#Convert date time string values into pandas datetime values
df['closed_at'] = pd.to_datetime(df['closed_at'])
df['opened_at'] = pd.to_datetime(df['opened_at'])
df['resolved_at'] = pd.to_datetime(df['resolved_at'])
df['sys_updated_at'] = pd.to_datetime(df['sys_updated_at'])


In [186]:
#calculate features like lifetime = time the incident was active, action_at = time taken when first action was undertaken
df['lifetime'] = df['closed_at'] - df['opened_at']
df['action_at'] = df['sys_updated_at'] - df['opened_at']

In [187]:
#extract only the day values from datetime value for features
df['lifetime'] = df['lifetime'].dt.days
df['action_at'] = df['action_at'].dt.days

In [188]:
#check number of records having close time before open time (which is wrong : It is because the the record opentime was updated by the user. Hence we extract the original open time by sorting cases with similar case number in ascending order and taking the opentime from the record with earliest system updated time)
df['number'][df['lifetime'] < 0], df['opened_at'][df['lifetime'] < 0]

(489       INC0000174
 490       INC0000174
 491       INC0000174
 492       INC0000174
 493       INC0000174
              ...    
 141665    INC0118952
 141666    INC0118952
 141667    INC0118999
 141668    INC0118999
 141669    INC0118999
 Name: number, Length: 50482, dtype: object,
 489      2016-02-29 10:39:00
 490      2016-02-29 10:39:00
 491      2016-02-29 10:39:00
 492      2016-02-29 10:39:00
 493      2016-02-29 10:39:00
                  ...        
 141665   2017-10-02 14:48:00
 141666   2017-10-02 14:48:00
 141667   2017-10-02 15:55:00
 141668   2017-10-02 15:55:00
 141669   2017-10-02 15:55:00
 Name: opened_at, Length: 50482, dtype: datetime64[ns])

In [189]:
def update_same_case_records(df):
    df.reset_index()
    unique_case_ids = df['number'].unique()
    if len(unique_case_ids) > 0:
        for case_id in tqdm(unique_case_ids):
            #indicidual case id reccords
            df_case = df[df['number'] == case_id]
            #sort the column by system updated at
            sorted_df_case = df_case.sort_values('sys_updated_at')
            #sorted_df_case.reset_index(inplace=True)
            initial_record = sorted_df_case.iloc[[0]]
            latest_record = sorted_df_case.iloc[[sorted_df_case.shape[0]-1]]
            lifetime = latest_record['closed_at'] - latest_record['opened_at']
            lifetime = lifetime.dt.days.values[0]
            if  lifetime <= 0: latest_record['opened_at'] = initial_record['sys_updated_at'].values
            df = df[df['number'] != case_id]
            if df['number'].isnull().values.any():
                df = pd.concat([df,latest_record.to_frame()])
            else: df = pd.concat([df,latest_record])
    return df


In [190]:
df_same_case = df[df['lifetime'] < 0]
df_unique_case = df[df['lifetime'] > 0]

In [191]:
df_same_case = update_same_case_records(df_same_case)

100%|██████████| 8614/8614 [03:31<00:00, 40.69it/s]


In [192]:
#merge unique cases and the newly created df having single record for every case number
df_new = pd.concat([df_unique_case,df_same_case])

In [193]:
# create features lifetime and action_at
df_new['lifetime'] = df_new['closed_at'] - df_new['opened_at']
df_new['action_at'] = df_new['sys_updated_at'] - df_new['opened_at']

#store only days values as features
df_new['lifetime'] = df_new['lifetime'].dt.days
df_new['action_at'] = df_new['action_at'].dt.days

#scale lifetime and action_at features
features = ['lifetime','action_at']
df_new[features] = scaler.fit_transform(df_new[features])

In [194]:
df_new.head()

,number,active,reassignment_count,reopen_count,sys_mod_count,made_sla,opened_at,sys_created_at,sys_updated_at,contact_type,...,closed_code,resolved_at,closed_at,incident_state_Active,incident_state_Awaiting,incident_state_Closed,incident_state_New,incident_state_Resolved,lifetime,action_at
0,INC0000045,0,1,0,0,0,2016-02-29 01:16:00,29/2/2016 01:23,2016-02-29 01:23:00,Phone,...,code 5,2016-02-29 11:29:00,2016-05-03 12:00:00,0,0,0,1,0,0.106136,0.335171
1,INC0000045,0,1,0,2,0,2016-02-29 01:16:00,29/2/2016 01:23,2016-02-29 08:53:00,Phone,...,code 5,2016-02-29 11:29:00,2016-05-03 12:00:00,0,0,0,0,1,0.106136,0.335171
2,INC0000045,0,1,0,3,0,2016-02-29 01:16:00,29/2/2016 01:23,2016-02-29 11:29:00,Phone,...,code 5,2016-02-29 11:29:00,2016-05-03 12:00:00,0,0,0,0,1,0.106136,0.335171
3,INC0000045,0,1,0,4,0,2016-02-29 01:16:00,29/2/2016 01:23,2016-05-03 12:00:00,Phone,...,code 5,2016-02-29 11:29:00,2016-05-03 12:00:00,0,0,1,0,0,0.106136,0.405733
4,INC0000047,0,1,0,0,0,2016-02-29 04:40:00,29/2/2016 04:57,2016-02-29 04:57:00,Phone,...,code 5,2016-01-03 09:52:00,2016-06-03 10:00:00,0,0,0,1,0,0.157546,0.335171


In [195]:
#check if there are records with negative lifetime
df_new['number'][df_new['lifetime'] < 0], df_new['opened_at'][df_new['lifetime'] < 0]

(Series([], Name: number, dtype: object),
 Series([], Name: opened_at, dtype: datetime64[ns]))

In [196]:
# store case numbers for analytical purpose
ids = df_new['number']
df_new.drop(['number'],inplace = True, axis = 1)

In [197]:
df_new.columns

Index(['active', 'reassignment_count', 'reopen_count', 'sys_mod_count',
       'made_sla', 'opened_at', 'sys_created_at', 'sys_updated_at',
       'contact_type', 'category', 'subcategory', 'u_symptom', 'impact',
       'urgency', 'priority', 'knowledge', 'u_priority_confirmation',
       'closed_code', 'resolved_at', 'closed_at', 'incident_state_Active',
       'incident_state_Awaiting', 'incident_state_Closed',
       'incident_state_New', 'incident_state_Resolved', 'lifetime',
       'action_at'],
      dtype='object')

In [198]:
# drop irrelevant features
df_new.drop(['closed_at','opened_at','sys_updated_at','sys_created_at','resolved_at','closed_code','category','subcategory','u_symptom','contact_type'],axis = 1,inplace=True)

In [199]:
# reset index values and drop index column for the model input
df_new.reset_index(inplace= True)
df_new.drop(['index'],axis = 1,inplace=True)

In [200]:
df_new.head()

,active,reassignment_count,reopen_count,sys_mod_count,made_sla,impact,urgency,priority,knowledge,u_priority_confirmation,incident_state_Active,incident_state_Awaiting,incident_state_Closed,incident_state_New,incident_state_Resolved,lifetime,action_at
0,0,1,0,0,0,2,2,3,0,0,0,0,0,1,0,0.106136,0.335171
1,0,1,0,2,0,2,2,3,0,0,0,0,0,0,1,0.106136,0.335171
2,0,1,0,3,0,2,2,3,0,0,0,0,0,0,1,0.106136,0.335171
3,0,1,0,4,0,2,2,3,0,0,0,0,1,0,0,0.106136,0.405733
4,0,1,0,0,0,2,2,3,0,0,0,0,0,1,0,0.157546,0.335171


In [1343]:
df_new.to_csv('Data/reopened.csv')